In [42]:

from ipywidgets import *
from IPython.display import display
import numpy as np
import math
    
################################
#Functions

def createLabel():
    return HTML(value="Made by <i>S.Tsotras</i>",)
    
#Numeric Text UI
def createText(description, numType):
    if numType == 'int':
        return IntText(description=description, layout=Layout(width='auto'))
    elif numType == 'float':
        return FloatText(description=description, layout=Layout(width='auto'))
    else:
        print('wrong Type')
        return Text(description=description, layout=Layout(width='auto'))


#Buttons UI
def createButton(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='50px', width='auto'))


#Layout Generator UI
def makeGenerator(i):
    generatorItems = [createText("Ug"+str(i), 'int'), createText("Sg"+str(i), 'int'), createText("xd"+str(i), 'float')]
    transformerItems = [createText("uk"+str(i), 'int'), createText("Utr"+str(i), 'int'), createText("Str"+str(i), 'int'), createText("Ptr"+str(i), 'int') ]

    return VBox(children=generatorItems + transformerItems, layout=Layout(border='0px solid gray', padding='10px'))


################################
#HEADER UI

buttonItems = [createButton("Add Generator", 'info'), createButton("Clear", 'warning')]
globalItems = [createText("c1", 'float'), createText("Ustation", 'int'), createText("Sstation", 'int')]

calButton = createButton("Calculate", 'success')


#Layout for Header
textUI = GridBox(globalItems, layout=widgets.Layout(grid_template_columns="repeat(3, 180px)"))
buttonsUI = HBox(children=[HBox(children=buttonItems, layout=Layout(padding='10px')), HBox([calButton], layout=Layout(padding='10px'))] , layout=Layout(width='300px', justify_content='space-between'))

headerUI = VBox(children=[buttonsUI , textUI],  layout=Layout(padding='10px'))


# Populate Global Items
headerUI.children[1].children[0].value = 1.1
headerUI.children[1].children[1].value = 20000
headerUI.children[1].children[2].value = 1000000



################################
#CENTER UI   

items = [makeGenerator(1), makeGenerator(2)]
centerUI = GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(3, 200px)"))


# Populate Generator Items
centerUI.children[0].children[0].value = 20000
centerUI.children[0].children[1].value = 618000
centerUI.children[0].children[2].value = 10.8


# Populate Transformer Items
centerUI.children[0].children[3].value = 6
centerUI.children[0].children[4].value = 20000
centerUI.children[0].children[5].value = 630000
centerUI.children[0].children[6].value = 7800


# Populate Generator Items
centerUI.children[1].children[0].value = 20000
centerUI.children[1].children[1].value = 618000
centerUI.children[1].children[2].value = 10.8


# Populate Transformer Items
centerUI.children[1].children[3].value = 6
centerUI.children[1].children[4].value = 20000
centerUI.children[1].children[5].value = 630000
centerUI.children[1].children[6].value = 7800


################################
#APP LAYOUT  

app = AppLayout(header=headerUI,
          left_sidebar=None,
          center=centerUI,
          right_sidebar=None,
          footer=None)

#Reactive output 
out = Output()
display(out)




################################
#Button Callbacks

def addGeneratorCallback(b):
    with out:
        items.append(makeGenerator(len(items)+1))
        app.center = GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(3, 200px)"))
        
def clearCallback(b):
    with out:
        items.clear()
        items.append(makeGenerator(1))
        app.center = GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(3, 200px)"))
        
        
def calculateCallback(b):
    with out:
        
        ####
        
        c1 = app.header.children[1].children[0].value  # voltage factor(page 22 of IEC60909)
        Ustation = app.header.children[1].children[1].value  # voltage of total substation
        Sstation = app.header.children[1].children[2].value  # apparent power of total substation
        
        
        ####
        
        
        N = len(app.center.children)  # number of generators
        ZHELP = 0  # initial input
        Xd = [0] * N  # necessary for matlab ( same for the other lines)
        Ztr = [0] * N
        Rtr = [0] * N
        Xtr = [0] * N
        R = [0] * N
        X = [0] * N
        Z = [0] * N
        ZI = [0] * N
        ZR = [0] * N
        ZHEL = [0] * N
        
        Ug = [0] * N  # voltage of generator 
        Sg = [0] * N # apparent power of generator 
        xd = [0] * N # Sub transient reactance of generator per unit
        uk = [0] * N # short circuit voltage of transformer 
        Utr = [0] * N # voltage of transformer 
        Str = [0] * N # apparent power of transformer 
        Ptr = [0] * N # losses of transformer 
        
        for i in range(N):
            
            Ug[i] = app.center.children[i].children[0].value
            Sg[i] = app.center.children[i].children[1].value   
            xd[i] = app.center.children[i].children[2].value 
            uk[i] = app.center.children[i].children[3].value  
            Utr[i] = app.center.children[i].children[4].value 
            Str[i] = app.center.children[i].children[5].value  
            Ptr[i] = app.center.children[i].children[6].value  
            
            Xd[i] = (xd[i]*Ug[i]*Ug[i])/(100*Sg[i]) #Sub traisieit reactaice of geierator i
            Ztr[i] = (uk[i]*Utr[i]*Utr[i])/(100*Str[i])
            Rtr[i] = (Ptr[i]*Utr[i]*Utr[i])/(Str[i]*Str[i])
            Xtr[i] = math.sqrt(Ztr[i]*Ztr[i]- Rtr[i]*Rtr[i]) #traisieit reactaice of traisformer i
            R[i] = Rtr[i] #resistaice of system traisformer1-geierator1
            X[i] = Xd[i]+Xtr[i] #reactaice of system traisformer1-geierator1
            Z[i]= R[i]+1j*X[i]
            ZHEL[i]= 1/(Z[i]) # ( geierally for parallel Z Zsyioliko= 1/(1/(Z1)+1/(Z2)+...)
            ZHELP= ZHELP + ZHEL[i] # ( geierally for parallel Z Zsyioliko= 1/(1/(Z1)+1/(Z2)+...)
            
        ZOL = 1 / ZHELP  # ( generally for parallel Z Zsynoliko= 1/(1/(Z1)+1/(Z2)+...)
        ZOLI = ZOL.imag
        ZOLR = ZOL.real

        Z = math.sqrt(ZOLI*ZOLI+ZOLR*ZOLR)  # total impedance of system
        Ik = (c1*Ustation)/(1.732*Z)  # short circuit current
        Scontribution = (Ik*1.732*Ustation)/(Sstation)  # contribution of subsystem intotal short-circuit current(< or = 8)
    
        
        print("Calculated Scontribution: ", Scontribution)
            
                
                
                

headerUI.children[0].children[0].children[0].on_click(addGeneratorCallback)
headerUI.children[0].children[0].children[1].on_click(clearCallback)

headerUI.children[0].children[1].children[0].on_click(calculateCallback)

display(app)

Output()

AppLayout(children=(VBox(children=(HBox(children=(HBox(children=(Button(button_style='info', description='Add …